In [1]:
from forecaster import TSForecaster
import pandas as pd

In [2]:
train = pd.read_csv("/Users/karnavivek/Food-Demand-Forecasting/Food Demand/data/train.csv")
meal_info = pd.read_csv("/Users/karnavivek/Food-Demand-Forecasting/Food Demand/data/meal_info.csv")
fulfilment_center_info = pd.read_csv("/Users/karnavivek/Food-Demand-Forecasting/Food Demand/data/fulfilment_center_info.csv")
test = pd.read_csv("/Users/karnavivek/Food-Demand-Forecasting/Food Demand/data/test.csv")

merge1 = pd.merge(train, fulfilment_center_info, how='left')
merge2 = pd.merge(merge1, meal_info, how='left')

merge2[['center_id', 'meal_id', 'emailer_for_promotion', 'homepage_featured']] = merge2[['center_id', 'meal_id', 'emailer_for_promotion', 'homepage_featured']].astype('object')

df_weekly_orders = merge2.groupby(['week'])['num_orders'].sum().reset_index()
df_weekly_orders.set_index(df_weekly_orders['week'], inplace=True)
df_weekly_orders.drop(columns='week', inplace=True)


In [3]:
data = pd.DataFrame(df_weekly_orders['num_orders'])
data.shape

(145, 1)

In [4]:
forecast = TSForecaster('arima')
forecast.stationarity_test(df_weekly_orders['num_orders'])

T-stat: -2.542865772667447
P Value: 0.10538983404031177
Lags: 9
Number of Observations used: 135
Critical Value at 1%: -3.479742586699182

Critical Value at 5%: -2.88319822181578

Critical Value at 10%: -2.578319684499314



'This Time Series Model is NON STATIONARY, may give bad predictions! :('

In [4]:
#as we saw above, the data is NON STATIONARY, so to transform the data into a STATIONARY one, we can use the following code:
new_data = forecast.make_stationary(df_weekly_orders['num_orders'], order=1)
new_data

0       -5177.0
1      -91822.0
2       48267.0
3      455146.0
4     -251387.0
         ...   
139    -26449.0
140    -36579.0
141     20265.0
142     41271.0
143   -129271.0
Name: num_orders, Length: 144, dtype: float64

In [ ]:
#lets perform stationarity test using the new transformed data
forecast.stationarity_test(new_data)

T-stat: -2.542865772667447
P Value: 0.10538983404031177
Lags: 9
Number of Observations used: 135
Critical Value at 1%: -3.479742586699182

Critical Value at 5%: -2.88319822181578

Critical Value at 10%: -2.578319684499314



'This Time Series Model is NON STATIONARY, may give bad predictions! :('

Following code is for LSTM test:

In [5]:
train_data = data[:115]
test_data = data[115:]

In [6]:
train_data.shape, test_data.shape

((115, 1), (30, 1))

In [7]:
lstm_predictions = forecast.run_models('lstm', train_data, test_data)
lstm_predictions

Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2355 - mean_absolute_error: 0.4629
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2337 - mean_absolute_error: 0.4610
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2316 - mean_absolute_error: 0.4588
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2297 - mean_absolute_error: 0.4567
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2276 - mean_absolute_error: 0.4545
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2260 - mean_absolute_error: 0.4528 
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2238 - mean_absolute_error: 0.4504
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2217 - mean_absolute_error: 0.4481
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2197 - mean_absolute_error: 0.4459
Epoch 10/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1650 - mean_absolute_error: 0.3890

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2171 - mean_absolute_error: 0.4430
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2152 - mean_absolute_error: 0.4409 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2123 - mean_absolute_error: 0.4376 
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2102 - mean_absolute_error: 0.4353
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2077 - mean_absolute_error: 0.4326 
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2052 - mean_absolute_error: 0.4298
Epoch 16/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2023 - mean_absolute_error: 0.4262
Epoch 17/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1988 - mean_absolute_error: 0.4222 
Epoch 18/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1958 - mean_absolute_error: 0.4188
Epoch 19/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1929 - mean_absolute_error: 0.4155
Epoch 20/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0

array([[802430.8 ],
       [816043.3 ],
       [834201.94],
       [848148.3 ],
       [850731.44],
       [849146.6 ],
       [830740.5 ],
       [806700.06],
       [805790.9 ],
       [808193.75],
       [830301.4 ],
       [843069.9 ],
       [867561.44],
       [860878.5 ],
       [880780.9 ],
       [858715.06],
       [839555.  ],
       [811590.94],
       [789047.6 ],
       [764254.06],
       [767170.1 ],
       [767348.7 ],
       [769973.56],
       [781063.3 ],
       [787028.5 ]], dtype=float32)